# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds, eigs
from sklearn.decomposition import TruncatedSVD
from numpy import dot
from numpy.linalg import norm

In [ ]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [5]:
config = pd.read_csv('config.csv').iloc[0]

In [6]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [27]:
user = config['id']
k_value = config['k']
user

276633

### Read the Data

In [55]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [29]:
# your code here

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [56]:
user_likes = dict()

In [57]:
images=set()
users=set()

In [58]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    images.add(item_id)
    users.add(user_id)
    user_likes[user_id].append(item_id)

In [59]:
len(images)

178788

In [60]:
users=list(users)
images=list(images)

In [61]:
user_indices=dict()
image_indices=dict()
for i in range(len(users)):
  user_indices[users[i]]=i

for i in range(len(images)):
  image_indices[images[i]]=i

In [62]:
rows=[]
cols=[]
values=[]
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])
    rows.append(user_indices[user_id])
    cols.append(image_indices[item_id])
values=[1]*len(rows)


In [63]:
len(rows)

1000000

In [64]:
rows=np.array(rows)
cols=np.array(cols)
values=np.array(values)

## Matrix Decomposition

In [65]:
cs_matrix=csr_matrix((values, (rows, cols))).tocsr()

In [66]:
cs_matrix.shape

(63497, 178788)

In [67]:
n_comp=1000
while True:
  global m
  svd = TruncatedSVD(n_components=n_comp, random_state=42)
  m=svd.fit(cs_matrix)
  print(svd.explained_variance_ratio_.sum())
  if(svd.explained_variance_ratio_.sum()>=0.8):
    break
  n_comp+=250

Note: Number of components depend on the explained variance ratio. Limited to the system specification, no_of_components is taken as 1000

In [68]:
svd = TruncatedSVD(n_components=1000, random_state=42)
m=svd.fit_transform(cs_matrix)

In [69]:
print(svd.n_features_in_)

178788


In [70]:
m.shape

(63497, 1000)

### Use KNN after Collaborative Filtering to find nearest neighbors

In [71]:
def Cosine_similarity(a,b):
  return abs(dot(a,b)/(norm(a)*norm(b)))
def Cosine_distance(a,b):
  return 1-Cosine_similarity(a,b)

In [72]:
def neighbors(user,k_value):
  user_index=user_indices[user]
  user_vec=m[user_index]
  distances=[]
  for i in range(len(m)):
    if i!=user_index:
      distances.append([i,Cosine_distance(user_vec,m[i])])
  distances.sort(key=lambda x:x[1])
  distances=distances[:k_value]
  k_neighbours=[]
  for i in range(k_value):
    index=distances[i][0]
    k_neighbours.append(users[index])
    print(distances[i][1])
  return k_neighbours



In [54]:
neighbors(user,k_value)

0.10308780285651797
0.10308780285667896
0.5594346221864541
0.5636005075337738
0.5950759477856786


[1973004, 1494939, 1480429, 2452817, 3660527]

## KNN using Jaccard

In [ ]:
# your code here
# your code here
# def neighbors_jaccard(user,k_value):
#     """ returns an iterable object (like list or generator) """
#     similarity_dict=dict()
#     current_user_likes=set(user_likes[user])
#     for u in user_likes:
#       if u==user:
#         continue
#       temp=set(user_likes[u])
#       aUnionb=current_user_likes.union(temp)
#       aintersectionb=current_user_likes.intersection(temp)
#       similarity_dict[u]=len(aintersectionb)

#     sorted_list=sorted(similarity_dict.items(),key=lambda item:item[1],reverse=True)
#     sorted_list=sorted_list[:k_value]
#     # print(sorted_list)
#     result=[u[0] for u in sorted_list]
#     return result

In [ ]:
# neighbors_jaccard(user,k_value)

[3628520, 666917, 2492899, 3313136, 2585131]

### Open the output file to write all the lines to the file

In [73]:
outFile = open(config['output_file'], 'w')

for user_id in neighbors(user,k_value):
    # user_id = list(user_likes.keys())[n_user]
    for item_id in user_likes[user_id]:
      if(item_id not in user_likes[user]):
        outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()

0.10308780285651797
0.10308780285667896
0.5594346221864541
0.5636005075337738
0.5950759477856786
